In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import auc
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore', message='Changing the shape of non-C contiguous array')

In [2]:
# Load the data
train_df = pd.read_csv('./OFFER/DATA/can_train_mas_CRCMRAPM.csv', header=0)
test_df = pd.read_csv('./OFFER/DATA/can_oot_mas_CRCMRAPM.csv', header=0)


#put all the independent variables w/0 key and dependent variable
columns=train_df.columns.tolist()
print columns
feature_columns_to_use = columns[2:]
print feature_columns_to_use

#put the non numeric variables in ' ' .eg 'off_code'
nonnumeric_columns = ['off_code']


big_X = train_df[feature_columns_to_use].append(test_df[feature_columns_to_use])

#handling non numeric columns
le = LabelEncoder()
for feature in nonnumeric_columns:
    big_X[feature] = le.fit_transform(big_X[feature])

# Prepare the inputs for the model
train_X = big_X[0:train_df.shape[0]].as_matrix()
test_X = big_X[train_df.shape[0]::].as_matrix()

#put the dependent variable
train_y = train_df['resp_new']
test_y  = test_df ['resp_new']



['extcmno', 'resp_new', 'off_code', 'TOTOPNR', 'AVBEACE', 'cocm12', 'tbgr1c', 'CUMMVEST', 'SOWTRNDB', 'NO_GOLD', 'CDSMSC60', 'mrind', 'recm12', 'cdlyage', 'lastspd', 'NACCINC', 'HIGHREV', 'tbym16', 'NPROMIN', 'air_spend_local_12mos_avg', 'cocv12', 'tbxm1c', 'tbtr13', 'tbcr1c', 'bfrr1c', 'CUAVPAYC']
['off_code', 'TOTOPNR', 'AVBEACE', 'cocm12', 'tbgr1c', 'CUMMVEST', 'SOWTRNDB', 'NO_GOLD', 'CDSMSC60', 'mrind', 'recm12', 'cdlyage', 'lastspd', 'NACCINC', 'HIGHREV', 'tbym16', 'NPROMIN', 'air_spend_local_12mos_avg', 'cocv12', 'tbxm1c', 'tbtr13', 'tbcr1c', 'bfrr1c', 'CUAVPAYC']


In [ ]:

gbm = xgb.XGBClassifier()
gbm_params = {
    'learning_rate': [ 0.009,0.01,0.02,0.03],
    'n_estimators': [600],
    'max_depth': [3]
}
cv = StratifiedKFold(train_y)
grid = GridSearchCV(gbm, gbm_params,scoring='roc_auc',cv=cv,verbose=10,n_jobs=10)
grid.fit(train_X, train_y)

print (grid.best_params_)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] n_estimators=600, learning_rate=0.009, max_depth=3 ..............
[CV] n_estimators=600, learning_rate=0.009, max_depth=3 ..............
[CV] n_estimators=600, learning_rate=0.009, max_depth=3 ..............
[CV] n_estimators=600, learning_rate=0.01, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.01, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.01, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.02, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.02, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.02, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.03, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.03, max_depth=3 ...............
[CV] n_estimators=600, learning_rate=0.03, max_depth=3 ...............
[CV]  n_estimators=600, learning_rate=0.03, max_depth=3, score=0.669291 -   6.4s
[CV]  n

[Parallel(n_jobs=10)]: Done   1 tasks      | elapsed:    7.2s
Process PoolWorker-22:
Process PoolWorker-21:
Process PoolWorker-24:
Traceback (most recent call last):
Process PoolWorker-26:
Process PoolWorker-27:
Traceback (most recent call last):
Process PoolWorker-23:
Process PoolWorker-25:
Process PoolWorker-28:
  File "/adshome/svjha1/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-29:
Process PoolWorker-30:
  File "/adshome/svjha1/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/adshome/svjha1/anaconda2/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/adshome/svjha1/anaconda2/lib/python2.7/multipr

In [3]:
gbm = xgb.XGBClassifier(max_depth=3, learning_rate=0.01, n_estimators=500)
gbm.fit(train_X, train_y)
y_pred = gbm.predict_proba(test_X)[:,1]
fpr, tpr, thresholds = metrics.roc_curve(test_y,y_pred,pos_label=1)
auc = metrics.auc(fpr, tpr)
gini=(2*auc-1)*100
print gini


0.669083032742
